## Padé approximants
Using the example of $\sin(x)$ for simplicity.

In [ ]:
from sympy import Symbol, sin, series, Poly, symbols, Eq, solve, Order, plot, pi

In [ ]:
x = Symbol("x")

In [ ]:
# n+m for the Padé approximant
m = 2
n = 2

In [ ]:
degree = m+n
taylorO = sin(x).series(x, 0, degree + 1)
taylorO

In [ ]:
# drop the O
taylor = taylorO.removeO()
taylor

In [ ]:
# coeff gives a coefficient but only for the specified term
taylor.coeff(x**3)

In [ ]:
# if we first convert to Poly, we can get all coefficients easily
Poly(taylor, x).all_coeffs()

We now solve 
$$\frac{A_0 + A_1 x + \dots + A_m x^m}{B_0 + B_1 x + \dots + B_n x^n} = \operatorname{Taylor}[f]$$

In [ ]:
# generate a list of symbols
As = symbols(f"A0:{m+1}")
Bs = symbols(f"B0:{m+1}")
As, Bs

In [ ]:
Poly(reversed(Bs), x) + 1 # SymPy doesn't know the domain of the Bs, so it adjoins them to the coefficient ring

In [ ]:
B0 = Bs[0]
B0 = 0 # doesn't work -- why?

In [ ]:
Bs1 = (1,) + Bs[1:]
Bs1

In [ ]:
poly_B = Poly(reversed(Bs1), x)
poly_B

In [ ]:
rhs_poly = poly_B * taylor
rhs_poly

In [ ]:
# rhs is still a polynomial. This doesn't have the expand method that solveset uses, so let's make it a simple expression:
rhs5 = rhs_poly.as_expr()
rhs5

In [ ]:
# we need to discard powers in x == m+n+1 -- many ways to do this, a simple one I found
rhs = (rhs5 + Order(x**(m+n+1))).removeO()
rhs

In [ ]:
# same for the left-hand side
(lhs := Poly(reversed(As), x).as_expr())

In [ ]:
eq = Eq(lhs, rhs)
eq

In [ ]:
sol_A = solve(eq, As)
sol_A

In [ ]:
eqA = eq.subs(sol_A)
eqA

In [ ]:
sol_B = solve(eqA, Bs1)
sol_B

In [ ]:
# now we have all coefficients and can put them into the original Padé approximant
pade = Poly(reversed(As), x)/Poly(reversed(Bs1), x)
pade

In [ ]:
pade_sin = pade.subs(sol_A).subs(sol_B)
pade_sin

Note: the approach above may not be the simplest nor best (in terms of generality). An alternative would be to extract the coefficients from the left- and right-hand side and construct a system of linear equations to be solved.

In [ ]:
p = plot(
    sin(x), taylor, pade_sin, 
    xlim = (0, 2*pi),
    ylim = (-2, 2),
    legend = True,
    show = False
)
# first want to update the legend before showing it 🤭
p[1].label = "Taylor"
p[2].label = "Padé"
p.show()

In [ ]:
# residuals
p = plot(
    0, taylor - sin(x), pade_sin - sin(x), 
    (x, 0, 4*pi), # range
    ylim = (-4, 4),
    legend = True,
    show = False
)
p[1].label = "Taylor"
p[2].label = "Padé"
p.show()

In [ ]:
# relative error 
p = plot(
    abs((pade_sin - sin(x)) / (taylor - sin(x))),
    (x, 0, 4*pi)
)